:warning:**IMPORTANT NOTICE**:warning:\
*Since the initial parameterisation and gradient descent optimisation are stochastic processes, the training of a neural network is not fully reproducible.*

*Therefore, it is not recommended to re-run this script as it will overwrite the original calibration of the neural network used in the work presented here.
The purpose of this script is solely to document the training procedure and can be copied as a template to fit other new neural networks.*

*To experiment with the models calibrated here, they can be loaded from the `saved_models` directory.*

# Systematic assesment of different hyperparameters to optimise model performance

Various hyperparameters are varied to asses their impact on the model performance in a semi-quantitative way.\
The hyperparameters that are varied are:
- Number of hidden layers / neurons --> model capacity
- Initial learning rate
- Activation function
- Effect of regularization

The model is trained on the same two datasets for each hyperparameter combination:
1. Biotite composition (FMAST)
2. Biotite composition (MnFMAST) + index minerals

### Model capacity

These architectures are tested:
- **very small**: 1 hidden layers with 8 neurons each
- **small**: 1 hidden layers with 16 neurons each
- **large**: 1 hidden layers with 32 neurons each
- **small_2hl**: 2 hidden layers with 16 neurons each
- **large_2hl**: 2 hidden layers with 32 neurons each
- **huge**: 3 hidden layers with 64 neurons

### Learning rate

These learning rates are tested:
- 0.01
- 0.001
- 0.0005
- 0.0001

### Activation function
*Decided against testing!*

### Regularisation

These regularisation methods are tested:
- L2 regularisation
- Dropout
- Batch normalisation
- Layer normalisation
- Combined L2 regularisation and dropout


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from pathlib import Path
from keras.models import Sequential
from keras.layers import Dense, Normalization, BatchNormalization, LayerNormalization, Dropout
from keras.losses import MeanSquaredError
from keras.optimizers import Adam, schedules
from keras.metrics import MeanAbsoluteError, RootMeanSquaredError
from keras.callbacks import CSVLogger, EarlyStopping
from sklearn.model_selection import train_test_split

from ml_tb.normalisation import MinMaxScaler
from ml_tb.metrics import RMSE_denormalised_P, RMSE_denormalised_T

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_18372\2311156246.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import data, pre-procesing and train/test split

Load the two datasets and split them into training and validation sets.

Validation set is **20%** of the training set. Approx. 200 samples.

In [2]:
VALIDATION_FRACTION = 0.2

For the scaling of the P-T data, a MinMaxScaler is used.
This scaler is defined globally and used for all datasets.

In [3]:
scaling_pt = MinMaxScaler(min=[1500, 400], max=[10000, 900], axis=0)
inv_scaling_pt = MinMaxScaler(min=[1500, 400], max=[10000, 900], axis=0, invert=True)

### **Dataset 1: Elements in biotite solution model (MnFMAST)**


In [4]:
# load excel file
data = pd.read_excel(Path("Metapelite-Database_Bt_CLEAN_2024-02-03.xlsx"))

biotite_composition = np.zeros(shape=(len(data), 6))
biotite_composition[:, 0] = data["Bt-Si"]
biotite_composition[:, 1] = data["Bt-Ti"]
biotite_composition[:, 2] = data["Bt-Al"]
biotite_composition[:, 3] = data["Bt-FeTot"]
biotite_composition[:, 4] = data["Bt-Mn"]
biotite_composition[:, 5] = data["Bt-Mg"]

pt = np.zeros(shape=(len(data), 2))
pt[:, 0] = data["Pressure estimate random uniform"] * 1000 # convert to bar
pt[:, 1] = data["Temperature random ordered after Ti-in-Bt"]

# check for NaN values (should be already filtered out)
print("NaN values in biotite composition: ", np.isnan(biotite_composition).any())
print("NaN values in PT: ", np.isnan(pt).any())

NaN values in biotite composition:  False
NaN values in PT:  False


In [5]:
# test train split
DATA01_biotite_composition_train, DATA01_biotite_composition_val, DATA01_pt_train, DATA01_pt_val = train_test_split(biotite_composition, pt, test_size=VALIDATION_FRACTION, shuffle=True)

DATA01_normalisation_biotite_composition = Normalization(axis=-1)
DATA01_normalisation_biotite_composition.adapt(DATA01_biotite_composition_train)

print(DATA01_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(DATA01_normalisation_biotite_composition.variance.numpy()))

DATA01_pt_train_norm = scaling_pt(DATA01_pt_train)
DATA01_pt_val_norm = scaling_pt(DATA01_pt_val)

print("After normalisation, the minimal value of P and T is: ", DATA01_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", DATA01_pt_train_norm.numpy().max(axis=0))

[[2.7081466  0.12082674 1.7106036  1.2264906  0.00852969 1.0918795 ]]
[[0.04705625 0.04727941 0.09257054 0.22646412 0.00741701 0.2587144 ]]
After normalisation, the minimal value of P and T is:  [0.00010304 0.00066034]
After normalisation, the maximal value of P and T is:  [0.940462  0.8988128]


### **Dataset 2: Elements in biotite solution model + Index minerals**


In [6]:
# load excel file
data = pd.read_excel(Path("Metapelite-Database_Bt_CLEAN_2024-02-03.xlsx"))

biotite_composition = np.zeros(shape=(len(data), 6))
biotite_composition[:, 0] = data["Bt-Si"]
biotite_composition[:, 1] = data["Bt-Ti"]
biotite_composition[:, 2] = data["Bt-Al"]
biotite_composition[:, 3] = data["Bt-FeTot"]
biotite_composition[:, 4] = data["Bt-Mn"]
biotite_composition[:, 5] = data["Bt-Mg"]

# extract one-hot encoded minerals in the following order: Chl, Grt, Crd, And, St, Ky, Sil, Kfs
index_minerals = np.zeros(shape=(len(data), 8))
index_minerals[:, 0] = data["Chl"]
index_minerals[:, 1] = data["Grt"]
index_minerals[:, 2] = data["Crd"]
index_minerals[:, 3] = data["And"]
index_minerals[:, 4] = data["St"]
index_minerals[:, 5] = data["Ky"]
index_minerals[:, 6] = data["Sil"]
index_minerals[:, 7] = data["Kfs"]

# Some minerals (Chl, Grt, St) have NaN values. Replace them with 0. Most likely samples with regional or metastable phases?!
index_minerals = np.nan_to_num(index_minerals, nan=0)

# combine biotite composition and one-hot encoded minerals
biotite_composition_idxmin = np.concatenate((biotite_composition, index_minerals), axis=1)


pt = np.zeros(shape=(len(data), 2))
pt[:, 0] = data["Pressure estimate random uniform"] * 1000 # convert to bar
pt[:, 1] = data["Temperature random ordered after Ti-in-Bt"]

# check for NaN values (should be already filtered out)
print("NaN values in biotite composition & index minerals: ", np.isnan(biotite_composition_idxmin).any())
print("NaN values in PT: ", np.isnan(pt).any())

NaN values in biotite composition & index minerals:  False
NaN values in PT:  False


In [7]:
# test train split
DATA02_biotite_composition_train, DATA02_biotite_composition_val, DATA02_pt_train, DATA02_pt_val = train_test_split(biotite_composition_idxmin, pt, test_size=VALIDATION_FRACTION, shuffle=True)

DATA02_normalisation_biotite_composition = Normalization(axis=-1)
DATA02_normalisation_biotite_composition.adapt(DATA02_biotite_composition_train)

# the one-hot encoded minerals should not be normalised. This is not necessary, as they are already in the range [0, 1]
# Set the mean and variance to 0 and 1, respectively
norm_mean = DATA02_normalisation_biotite_composition.mean.numpy()
norm_mean[0,-8:] = 0
norm_var = DATA02_normalisation_biotite_composition.variance.numpy()
norm_var[0,-8:] = 1
DATA02_normalisation_biotite_composition.mean = tf.convert_to_tensor(norm_mean)
DATA02_normalisation_biotite_composition.variance = tf.convert_to_tensor(norm_var)

print(DATA02_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(DATA02_normalisation_biotite_composition.variance.numpy()))

DATA02_pt_train_norm = scaling_pt(DATA02_pt_train)
DATA02_pt_val_norm = scaling_pt(DATA02_pt_val)

print("After normalisation, the minimal value of P and T is: ", DATA02_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", DATA02_pt_train_norm.numpy().max(axis=0))

[[2.7076685  0.12076807 1.7106233  1.2272779  0.00841155 1.0923196
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]
[[0.04731337 0.04801339 0.09337769 0.23137051 0.00724167 0.26396626
  1.         1.         1.         1.         1.         1.
  1.         1.        ]]
After normalisation, the minimal value of P and T is:  [0.00010304 0.00066034]
After normalisation, the maximal value of P and T is:  [0.940462  0.8988128]


## Set-up global training parameters

Define a function to calculate RMSE for pressure and temperature for unscaled values to have an interpretable metric.

All models are trained for a maximum of 5000 epochs.\
Early stopping is used with a patience (no improvement of val loss) of 50 epochs.\
Inverse time learning rate decay is used for all models.

In [8]:
def RMSE_P(y_true, y_pred):
    return RMSE_denormalised_P(y_true, y_pred, inv_scaling_pt)


def RMSE_T(y_true, y_pred):
    return RMSE_denormalised_T(y_true, y_pred, inv_scaling_pt)

In [9]:
BATCH_SIZE = 50
STEPS_PER_EPOCH = len(DATA01_biotite_composition_train) // BATCH_SIZE
MAX_EPOCHS = 5000

lr_schedule = schedules.InverseTimeDecay(0.001, decay_steps=STEPS_PER_EPOCH*1000, decay_rate=1, staircase=False)

LOSS = MeanSquaredError()
METRICS = [MeanAbsoluteError(), RootMeanSquaredError(), RMSE_P, RMSE_T]

## **Test 01:** Model capacity

- **very small**: 1 hidden layers with 8 neurons each
- **small**: 1 hidden layers with 16 neurons each
- **large**: 1 hidden layers with 32 neurons each
- **small_2hl**: 2 hidden layers with 16 neurons each
- **large_2hl**: 2 hidden layers with 32 neurons each
- **huge**: 3 hidden layers with 64 neurons

In [10]:
CALLBACKS_VERYSMALL_01 = [CSVLogger("HyperParamTest_verysmall_DATA01.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_VERYSMALL_02 = [CSVLogger("HyperParamTest_verysmall_DATA02.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_SMALL_01 = [CSVLogger("HyperParamTest_small_DATA01.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_SMALL_02 = [CSVLogger("HyperParamTest_small_DATA02.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LARGE_01 = [CSVLogger("HyperParamTest_large_DATA01.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LARGE_02 = [CSVLogger("HyperParamTest_large_DATA02.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_SMALL_2HL_01 = [CSVLogger("HyperParamTest_small_2HL_DATA01.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_SMALL_2HL_02 = [CSVLogger("HyperParamTest_small_2HL_DATA02.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LARGE_2HL_01 = [CSVLogger("HyperParamTest_large_2HL_DATA01.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LARGE_2HL_02 = [CSVLogger("HyperParamTest_large_2HL_DATA02.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_HUGE_01 = [CSVLogger("HyperParamTest_huge_DATA01.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_HUGE_02 = [CSVLogger("HyperParamTest_huge_DATA02.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [11]:
OPT = Adam(lr_schedule)

verysmall_model = Sequential()
verysmall_model.add(DATA01_normalisation_biotite_composition)
verysmall_model.add(Dense(8, activation="relu"))
verysmall_model.add(Dense(2))

verysmall_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
verysmall_model.summary()

history = verysmall_model.fit(DATA01_biotite_composition_train, DATA01_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA01_biotite_composition_val, DATA01_pt_val_norm],
                          callbacks=CALLBACKS_VERYSMALL_01, verbose=False)
verysmall_model.save(Path("saved_models", "verysmall_model_DATA01"))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense (Dense)               (None, 8)                 56        
                                                                 
 dense_1 (Dense)             (None, 2)                 18        
                                                                 
Total params: 87 (352.00 Byte)
Trainable params: 74 (296.00 Byte)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________




INFO:tensorflow:Assets written to: saved_models\verysmall_model_DATA01\assets


INFO:tensorflow:Assets written to: saved_models\verysmall_model_DATA01\assets


In [12]:
OPT = Adam(lr_schedule)

verysmall_model = Sequential()
verysmall_model.add(DATA02_normalisation_biotite_composition)
verysmall_model.add(Dense(8, activation="relu"))
verysmall_model.add(Dense(2))

verysmall_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
verysmall_model.summary()

history = verysmall_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                          callbacks=CALLBACKS_VERYSMALL_02, verbose=False)
verysmall_model.save(Path("saved_models", "verysmall_model_DATA02"))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_2 (Dense)             (None, 8)                 120       
                                                                 
 dense_3 (Dense)             (None, 2)                 18        
                                                                 
Total params: 167 (672.00 Byte)
Trainable params: 138 (552.00 Byte)
Non-trainable params: 29 (120.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\verysmall_model_DATA02\assets


INFO:tensorflow:Assets written to: saved_models\verysmall_model_DATA02\assets


In [13]:
OPT = Adam(lr_schedule)

small_model = Sequential()
small_model.add(DATA01_normalisation_biotite_composition)
small_model.add(Dense(16, activation="relu"))
small_model.add(Dense(2))

small_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
small_model.summary()

history = small_model.fit(DATA01_biotite_composition_train, DATA01_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA01_biotite_composition_val, DATA01_pt_val_norm],
                          callbacks=CALLBACKS_SMALL_01, verbose=False)
small_model.save(Path("saved_models", "small_model_DATA01"))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_4 (Dense)             (None, 16)                112       
                                                                 
 dense_5 (Dense)             (None, 2)                 34        
                                                                 
Total params: 159 (640.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\small_model_DATA01\assets


INFO:tensorflow:Assets written to: saved_models\small_model_DATA01\assets


In [14]:
OPT = Adam(lr_schedule)

small_model = Sequential()
small_model.add(DATA02_normalisation_biotite_composition)
small_model.add(Dense(16, activation="relu"))
small_model.add(Dense(2))

small_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
small_model.summary()

history = small_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                          callbacks=CALLBACKS_SMALL_02, verbose=False)
small_model.save(Path("saved_models", "small_model_DATA02"))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_6 (Dense)             (None, 16)                240       
                                                                 
 dense_7 (Dense)             (None, 2)                 34        
                                                                 
Total params: 303 (1.19 KB)
Trainable params: 274 (1.07 KB)
Non-trainable params: 29 (120.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\small_model_DATA02\assets


INFO:tensorflow:Assets written to: saved_models\small_model_DATA02\assets


In [15]:
OPT = Adam(lr_schedule)

small_2hl_model = Sequential()
small_2hl_model.add(DATA01_normalisation_biotite_composition)
small_2hl_model.add(Dense(16, activation="relu"))
small_2hl_model.add(Dense(16, activation="relu"))
small_2hl_model.add(Dense(2))

small_2hl_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
small_2hl_model.summary()

history = small_2hl_model.fit(DATA01_biotite_composition_train, DATA01_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA01_biotite_composition_val, DATA01_pt_val_norm],
                          callbacks=CALLBACKS_SMALL_2HL_01, verbose=False)
small_2hl_model.save(Path("saved_models", "small_2hl_model_DATA01"))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_8 (Dense)             (None, 16)                112       
                                                                 
 dense_9 (Dense)             (None, 16)                272       
                                                                 
 dense_10 (Dense)            (None, 2)                 34        
                                                                 
Total params: 431 (1.69 KB)
Trainable params: 418 (1.63 KB)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\small_2hl_model_DATA01\assets


INFO:tensorflow:Assets written to: saved_models\small_2hl_model_DATA01\assets


In [16]:
OPT = Adam(lr_schedule)

small_2hl_model = Sequential()
small_2hl_model.add(DATA02_normalisation_biotite_composition)
small_2hl_model.add(Dense(16, activation="relu"))
small_2hl_model.add(Dense(16, activation="relu"))
small_2hl_model.add(Dense(2))

small_2hl_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
small_2hl_model.summary()

history = small_2hl_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                          callbacks=CALLBACKS_SMALL_2HL_02, verbose=False)
small_2hl_model.save(Path("saved_models", "small_2hl_model_DATA02"))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_11 (Dense)            (None, 16)                240       
                                                                 
 dense_12 (Dense)            (None, 16)                272       
                                                                 
 dense_13 (Dense)            (None, 2)                 34        
                                                                 
Total params: 575 (2.25 KB)
Trainable params: 546 (2.13 KB)
Non-trainable params: 29 (120.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\small_2hl_model_DATA02\assets


INFO:tensorflow:Assets written to: saved_models\small_2hl_model_DATA02\assets


In [17]:
OPT = Adam(lr_schedule)

large_model = Sequential()
large_model.add(DATA01_normalisation_biotite_composition)
large_model.add(Dense(32, activation="relu"))
large_model.add(Dense(2))

large_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
large_model.summary()

history = large_model.fit(DATA01_biotite_composition_train, DATA01_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA01_biotite_composition_val, DATA01_pt_val_norm],
                          callbacks=CALLBACKS_LARGE_01, verbose=False)
large_model.save(Path("saved_models", "large_model_DATA01"))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_14 (Dense)            (None, 32)                224       
                                                                 
 dense_15 (Dense)            (None, 2)                 66        
                                                                 
Total params: 303 (1.19 KB)
Trainable params: 290 (1.13 KB)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\large_model_DATA01\assets


INFO:tensorflow:Assets written to: saved_models\large_model_DATA01\assets


In [18]:
OPT = Adam(lr_schedule)

large_model = Sequential()
large_model.add(DATA02_normalisation_biotite_composition)
large_model.add(Dense(32, activation="relu"))
large_model.add(Dense(2))

large_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
large_model.summary()

history = large_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                          callbacks=CALLBACKS_LARGE_02, verbose=False)
large_model.save(Path("saved_models", "large_model_DATA02"))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_16 (Dense)            (None, 32)                480       
                                                                 
 dense_17 (Dense)            (None, 2)                 66        
                                                                 
Total params: 575 (2.25 KB)
Trainable params: 546 (2.13 KB)
Non-trainable params: 29 (120.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\large_model_DATA02\assets


INFO:tensorflow:Assets written to: saved_models\large_model_DATA02\assets


In [19]:
OPT = Adam(lr_schedule)

large_2hl_model = Sequential()
large_2hl_model.add(DATA01_normalisation_biotite_composition)
large_2hl_model.add(Dense(32, activation="relu"))
large_2hl_model.add(Dense(2))

large_2hl_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
large_2hl_model.summary()

history = large_2hl_model.fit(DATA01_biotite_composition_train, DATA01_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA01_biotite_composition_val, DATA01_pt_val_norm],
                          callbacks=CALLBACKS_LARGE_2HL_01, verbose=False)
large_2hl_model.save(Path("saved_models", "large_2hl_model_DATA01"))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_18 (Dense)            (None, 32)                224       
                                                                 
 dense_19 (Dense)            (None, 2)                 66        
                                                                 
Total params: 303 (1.19 KB)
Trainable params: 290 (1.13 KB)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\large_2hl_model_DATA01\assets


INFO:tensorflow:Assets written to: saved_models\large_2hl_model_DATA01\assets


In [20]:
OPT = Adam(lr_schedule)

large_2hl_model = Sequential()
large_2hl_model.add(DATA02_normalisation_biotite_composition)
large_2hl_model.add(Dense(32, activation="relu"))
large_2hl_model.add(Dense(2))

large_2hl_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
large_2hl_model.summary()

history = large_2hl_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                          callbacks=CALLBACKS_LARGE_2HL_02, verbose=False)
large_2hl_model.save(Path("saved_models", "large_2hl_model_DATA02"))

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_20 (Dense)            (None, 32)                480       
                                                                 
 dense_21 (Dense)            (None, 2)                 66        
                                                                 
Total params: 575 (2.25 KB)
Trainable params: 546 (2.13 KB)
Non-trainable params: 29 (120.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\large_2hl_model_DATA02\assets


INFO:tensorflow:Assets written to: saved_models\large_2hl_model_DATA02\assets


In [21]:
OPT = Adam(lr_schedule)

huge_model = Sequential()
huge_model.add(DATA01_normalisation_biotite_composition)
huge_model.add(Dense(64, activation="relu"))
huge_model.add(Dense(64, activation="relu"))
huge_model.add(Dense(64, activation="relu"))
huge_model.add(Dense(2))

huge_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
huge_model.summary()

history = huge_model.fit(DATA01_biotite_composition_train, DATA01_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA01_biotite_composition_val, DATA01_pt_val_norm],
                          callbacks=CALLBACKS_HUGE_01, verbose=False)
huge_model.save(Path("saved_models", "huge_model_DATA01"))

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_22 (Dense)            (None, 64)                448       
                                                                 
 dense_23 (Dense)            (None, 64)                4160      
                                                                 
 dense_24 (Dense)            (None, 64)                4160      
                                                                 
 dense_25 (Dense)            (None, 2)                 130       
                                                                 
Total params: 8911 (34.81 KB)
Trainable params: 8898 (34.76 KB)
Non-trainable params: 13 (56.00 Byte)
_________________

INFO:tensorflow:Assets written to: saved_models\huge_model_DATA01\assets


In [22]:
OPT = Adam(lr_schedule)

huge_model = Sequential()
huge_model.add(DATA02_normalisation_biotite_composition)
huge_model.add(Dense(64, activation="relu"))
huge_model.add(Dense(64, activation="relu"))
huge_model.add(Dense(64, activation="relu"))
huge_model.add(Dense(2))

huge_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
huge_model.summary()

history = huge_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                          callbacks=CALLBACKS_HUGE_02, verbose=False)
huge_model.save(Path("saved_models", "huge_model_DATA02"))

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_26 (Dense)            (None, 64)                960       
                                                                 
 dense_27 (Dense)            (None, 64)                4160      
                                                                 
 dense_28 (Dense)            (None, 64)                4160      
                                                                 
 dense_29 (Dense)            (None, 2)                 130       
                                                                 
Total params: 9439 (36.88 KB)
Trainable params: 9410 (36.76 KB)
Non-trainable params: 29 (120.00 Byte)
________________

INFO:tensorflow:Assets written to: saved_models\huge_model_DATA02\assets


## **Test 02:** Learning rate

All tests are performed with the optimal model architecture from Test 01. --> *small*

These learning rates are tested:
- 0.01
- 0.001
- 0.0005
- 0.0001


In [23]:
CALLBACKS_LR1_01 = [CSVLogger("HyperParamTest_LR1_DATA01.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LR1_02 = [CSVLogger("HyperParamTest_LR1_DATA02.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LR2_01 = [CSVLogger("HyperParamTest_LR2_DATA01.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LR2_02 = [CSVLogger("HyperParamTest_LR2_DATA02.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LR3_01 = [CSVLogger("HyperParamTest_LR3_DATA01.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LR3_02 = [CSVLogger("HyperParamTest_LR3_DATA02.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LR4_01 = [CSVLogger("HyperParamTest_LR4_DATA01.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LR4_02 = [CSVLogger("HyperParamTest_LR4_DATA02.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [24]:
lr_schedule.initial_learning_rate = 0.01

In [25]:
OPT = Adam(lr_schedule)

LR1_model = Sequential()
LR1_model.add(DATA01_normalisation_biotite_composition)
LR1_model.add(Dense(16, activation="relu"))
LR1_model.add(Dense(2))

LR1_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
LR1_model.summary()

history = LR1_model.fit(DATA01_biotite_composition_train, DATA01_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA01_biotite_composition_val, DATA01_pt_val_norm],
                          callbacks=CALLBACKS_LR1_01, verbose=False)
LR1_model.save(Path("saved_models", "LR1_model_DATA01"))

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_30 (Dense)            (None, 16)                112       
                                                                 
 dense_31 (Dense)            (None, 2)                 34        
                                                                 
Total params: 159 (640.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\LR1_model_DATA01\assets


INFO:tensorflow:Assets written to: saved_models\LR1_model_DATA01\assets


In [26]:
OPT = Adam(lr_schedule)

LR1_model = Sequential()
LR1_model.add(DATA02_normalisation_biotite_composition)
LR1_model.add(Dense(16, activation="relu"))
LR1_model.add(Dense(2))

LR1_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
LR1_model.summary()

history = LR1_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                        batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                        validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                        callbacks=CALLBACKS_LR1_02, verbose=False)
LR1_model.save(Path("saved_models", "LR1_model_DATA02"))

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_32 (Dense)            (None, 16)                240       
                                                                 
 dense_33 (Dense)            (None, 2)                 34        
                                                                 
Total params: 303 (1.19 KB)
Trainable params: 274 (1.07 KB)
Non-trainable params: 29 (120.00 Byte)
_________________________________________________________________
INFO:tensorflow:Assets written to: saved_models\LR1_model_DATA02\assets


INFO:tensorflow:Assets written to: saved_models\LR1_model_DATA02\assets


In [27]:
lr_schedule.initial_learning_rate = 0.001

In [28]:
OPT = Adam(lr_schedule)

LR2_model = Sequential()
LR2_model.add(DATA01_normalisation_biotite_composition)
LR2_model.add(Dense(16, activation="relu"))
LR2_model.add(Dense(2))

LR2_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
LR2_model.summary()

history = LR2_model.fit(DATA01_biotite_composition_train, DATA01_pt_train_norm,
                        batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                        validation_data=[DATA01_biotite_composition_val, DATA01_pt_val_norm],
                        callbacks=CALLBACKS_LR2_01, verbose=False)

LR2_model.save(Path("saved_models", "LR2_model_DATA01"))

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_34 (Dense)            (None, 16)                112       
                                                                 
 dense_35 (Dense)            (None, 2)                 34        
                                                                 
Total params: 159 (640.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\LR2_model_DATA01\assets


INFO:tensorflow:Assets written to: saved_models\LR2_model_DATA01\assets


In [29]:
OPT = Adam(lr_schedule)

LR2_model = Sequential()
LR2_model.add(DATA02_normalisation_biotite_composition)
LR2_model.add(Dense(16, activation="relu"))
LR2_model.add(Dense(2))

LR2_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
LR2_model.summary()

history = LR2_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                        batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                        validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                        callbacks=CALLBACKS_LR2_02, verbose=False)

LR2_model.save(Path("saved_models", "LR2_model_DATA02"))

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_36 (Dense)            (None, 16)                240       
                                                                 
 dense_37 (Dense)            (None, 2)                 34        
                                                                 
Total params: 303 (1.19 KB)
Trainable params: 274 (1.07 KB)
Non-trainable params: 29 (120.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\LR2_model_DATA02\assets


INFO:tensorflow:Assets written to: saved_models\LR2_model_DATA02\assets


In [30]:
lr_schedule.initial_learning_rate = 0.0005

In [31]:
OPT = Adam(lr_schedule)

LR3_model = Sequential()
LR3_model.add(DATA01_normalisation_biotite_composition)
LR3_model.add(Dense(16, activation="relu"))
LR3_model.add(Dense(2))

LR3_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
LR3_model.summary()

history = LR3_model.fit(DATA01_biotite_composition_train, DATA01_pt_train_norm,
                        batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                        validation_data=[DATA01_biotite_composition_val, DATA01_pt_val_norm],
                        callbacks=CALLBACKS_LR3_01, verbose=False)

LR3_model.save(Path("saved_models", "LR3_model_DATA01"))

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_38 (Dense)            (None, 16)                112       
                                                                 
 dense_39 (Dense)            (None, 2)                 34        
                                                                 
Total params: 159 (640.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\LR3_model_DATA01\assets


INFO:tensorflow:Assets written to: saved_models\LR3_model_DATA01\assets


In [32]:
OPT = Adam(lr_schedule)

LR3_model = Sequential()
LR3_model.add(DATA02_normalisation_biotite_composition)
LR3_model.add(Dense(16, activation="relu"))
LR3_model.add(Dense(2))

LR3_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
LR3_model.summary()

history = LR3_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                        batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                        validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                        callbacks=CALLBACKS_LR3_02, verbose=False)

LR3_model.save(Path("saved_models", "LR3_model_DATA02"))

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_40 (Dense)            (None, 16)                240       
                                                                 
 dense_41 (Dense)            (None, 2)                 34        
                                                                 
Total params: 303 (1.19 KB)
Trainable params: 274 (1.07 KB)
Non-trainable params: 29 (120.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\LR3_model_DATA02\assets


INFO:tensorflow:Assets written to: saved_models\LR3_model_DATA02\assets


In [33]:
lr_schedule.initial_learning_rate = 0.0001

In [34]:
OPT = Adam(lr_schedule)

LR4_model = Sequential()
LR4_model.add(DATA01_normalisation_biotite_composition)
LR4_model.add(Dense(16, activation="relu"))
LR4_model.add(Dense(2))

LR4_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
LR4_model.summary()

history = LR4_model.fit(DATA01_biotite_composition_train, DATA01_pt_train_norm,
                        batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                        validation_data=[DATA01_biotite_composition_val, DATA01_pt_val_norm],
                        callbacks=CALLBACKS_LR4_01, verbose=False)

LR4_model.save(Path("saved_models", "LR4_model_DATA01"))

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_42 (Dense)            (None, 16)                112       
                                                                 
 dense_43 (Dense)            (None, 2)                 34        
                                                                 
Total params: 159 (640.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


INFO:tensorflow:Assets written to: saved_models\LR4_model_DATA01\assets


INFO:tensorflow:Assets written to: saved_models\LR4_model_DATA01\assets


In [35]:
OPT = Adam(lr_schedule)

LR4_model = Sequential()
LR4_model.add(DATA02_normalisation_biotite_composition)
LR4_model.add(Dense(16, activation="relu"))
LR4_model.add(Dense(2))

LR4_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
LR4_model.summary()

history = LR4_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                        batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                        validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                        callbacks=CALLBACKS_LR4_02, verbose=False)

LR4_model.save(Path("saved_models", "LR4_model_DATA02"))

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_44 (Dense)            (None, 16)                240       
                                                                 
 dense_45 (Dense)            (None, 2)                 34        
                                                                 
Total params: 303 (1.19 KB)
Trainable params: 274 (1.07 KB)
Non-trainable params: 29 (120.00 Byte)
_________________________________________________________________
INFO:tensorflow:Assets written to: saved_models\LR4_model_DATA02\assets


INFO:tensorflow:Assets written to: saved_models\LR4_model_DATA02\assets


## **Test 03:** Regularisation

Test if regularisation can improve performance if applied to models with the best training set performance determined in tests 01 and 02.\
- Model architecture: huge. 3 hidden layers with 64 neurons each.
- Dataset: Bt composition + index minerals.

In [36]:
# RESER learning rate to 0.001
lr_schedule.initial_learning_rate = 0.001

In [37]:
CALLBACKS_noReg = [CSVLogger("HyperParamTest_noReg.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_L2 = [CSVLogger("HyperParamTest_L2.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_dropout = [CSVLogger("HyperParamTest_dropout.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_BatchNorm = [CSVLogger("HyperParamTest_BatchNorm.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LayerNorm = [CSVLogger("HyperParamTest_LayerNorm.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_L2_dropout = [CSVLogger("HyperParamTest_L2_dropout.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [38]:
OPT = Adam(lr_schedule)

noReg_model = Sequential()
noReg_model.add(DATA02_normalisation_biotite_composition)
noReg_model.add(Dense(64, activation="relu"))
noReg_model.add(Dense(64, activation="relu"))
noReg_model.add(Dense(64, activation="relu"))
noReg_model.add(Dense(2))

noReg_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
noReg_model.summary()

history = noReg_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                          callbacks=CALLBACKS_noReg, verbose=False)
noReg_model.save(Path("saved_models", "noReg_model"))

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_46 (Dense)            (None, 64)                960       
                                                                 
 dense_47 (Dense)            (None, 64)                4160      
                                                                 
 dense_48 (Dense)            (None, 64)                4160      
                                                                 
 dense_49 (Dense)            (None, 2)                 130       
                                                                 
Total params: 9439 (36.88 KB)
Trainable params: 9410 (36.76 KB)
Non-trainable params: 29 (120.00 Byte)
________________

INFO:tensorflow:Assets written to: saved_models\noReg_model\assets


INFO:tensorflow:Assets written to: saved_models\noReg_model\assets


In [39]:
OPT = Adam(lr_schedule)

L2_model = Sequential()
L2_model.add(DATA02_normalisation_biotite_composition)
L2_model.add(Dense(64, activation="relu", kernel_regularizer="l2"))
L2_model.add(Dense(64, activation="relu", kernel_regularizer="l2"))
L2_model.add(Dense(64, activation="relu", kernel_regularizer="l2"))
L2_model.add(Dense(2))

L2_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
L2_model.summary()

history = L2_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                          callbacks=CALLBACKS_L2, verbose=False)

L2_model.save(Path("saved_models", "L2_model"))

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_50 (Dense)            (None, 64)                960       
                                                                 
 dense_51 (Dense)            (None, 64)                4160      
                                                                 
 dense_52 (Dense)            (None, 64)                4160      
                                                                 
 dense_53 (Dense)            (None, 2)                 130       
                                                                 
Total params: 9439 (36.88 KB)
Trainable params: 9410 (36.76 KB)
Non-trainable params: 29 (120.00 Byte)


_________________________________________________________________
INFO:tensorflow:Assets written to: saved_models\L2_model\assets


INFO:tensorflow:Assets written to: saved_models\L2_model\assets


In [40]:
OPT = Adam(lr_schedule)

dropout_model = Sequential()
dropout_model.add(DATA02_normalisation_biotite_composition)
dropout_model.add(Dense(64, activation="relu"))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(64, activation="relu"))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(64, activation="relu"))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(2))

dropout_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
dropout_model.summary()

history = dropout_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                         batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                         validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                         callbacks=CALLBACKS_dropout, verbose=False)
dropout_model.save(Path("saved_models", "dropout_model"))

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_54 (Dense)            (None, 64)                960       
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_55 (Dense)            (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_56 (Dense)            (None, 64)                4160      
                                                     

INFO:tensorflow:Assets written to: saved_models\dropout_model\assets


In [41]:
OPT = Adam(lr_schedule)

BatchNorm_model = Sequential()
BatchNorm_model.add(DATA02_normalisation_biotite_composition)
BatchNorm_model.add(Dense(64, activation="relu"))
BatchNorm_model.add(BatchNormalization())
BatchNorm_model.add(Dense(64, activation="relu"))
BatchNorm_model.add(BatchNormalization())
BatchNorm_model.add(Dense(64, activation="relu"))
BatchNorm_model.add(BatchNormalization())
BatchNorm_model.add(Dense(2))

BatchNorm_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
BatchNorm_model.summary()

history = BatchNorm_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                              batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                              validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                              callbacks=CALLBACKS_BatchNorm, verbose=False)
BatchNorm_model.save(Path("saved_models", "BatchNorm_model"))

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_58 (Dense)            (None, 64)                960       
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dense_59 (Dense)            (None, 64)                4160      
                                                                 
 batch_normalization_1 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                     

INFO:tensorflow:Assets written to: saved_models\BatchNorm_model\assets


In [42]:
OPT = Adam(lr_schedule)

LayerNorm_model = Sequential()
LayerNorm_model.add(DATA02_normalisation_biotite_composition)
LayerNorm_model.add(Dense(64, activation="relu"))
LayerNorm_model.add(LayerNormalization())
LayerNorm_model.add(Dense(64, activation="relu"))
LayerNorm_model.add(LayerNormalization())
LayerNorm_model.add(Dense(64, activation="relu"))
LayerNorm_model.add(LayerNormalization())
LayerNorm_model.add(Dense(2))

LayerNorm_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
LayerNorm_model.summary()

history = LayerNorm_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                              batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                              validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                              callbacks=CALLBACKS_LayerNorm, verbose=False)
LayerNorm_model.save(Path("saved_models", "LayerNorm_model"))

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_62 (Dense)            (None, 64)                960       
                                                                 
 layer_normalization (Layer  (None, 64)                128       
 Normalization)                                                  
                                                                 
 dense_63 (Dense)            (None, 64)                4160      
                                                                 
 layer_normalization_1 (Lay  (None, 64)                128       
 erNormalization)                                                
                                                     

INFO:tensorflow:Assets written to: saved_models\LayerNorm_model\assets


In [43]:
OPT = Adam(lr_schedule)

l2_dropout_model = Sequential()
l2_dropout_model.add(DATA02_normalisation_biotite_composition)
l2_dropout_model.add(Dense(64, activation="relu", kernel_regularizer="l2"))
l2_dropout_model.add(Dropout(0.2))
l2_dropout_model.add(Dense(64, activation="relu", kernel_regularizer="l2"))
l2_dropout_model.add(Dropout(0.2))
l2_dropout_model.add(Dense(64, activation="relu", kernel_regularizer="l2"))
l2_dropout_model.add(Dropout(0.2))
l2_dropout_model.add(Dense(2))

l2_dropout_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
l2_dropout_model.summary()

history = l2_dropout_model.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                              batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                              validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                              callbacks=CALLBACKS_L2_dropout, verbose=False)
l2_dropout_model.save(Path("saved_models", "L2_dropout_model"))

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_66 (Dense)            (None, 64)                960       
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_67 (Dense)            (None, 64)                4160      
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_68 (Dense)            (None, 64)                4160      
                                                     

INFO:tensorflow:Assets written to: saved_models\L2_dropout_model\assets
